In [ ]:
import time
import random
import pandas as pd
from typing import List, Dict, Any
from datetime import date, timedelta
from pytrends.request import TrendReq

# ---------------------------
# Pytrends 초기화
# ---------------------------
_PT = TrendReq(
    hl="ko-KR",
    tz=540,
    retries=3,
    backoff_factor=0.1,
)

# ============================================================
#  Google Trends - 오늘 기준 정확한 1년 범위
# ============================================================
def get_google_trends_1year(
    keyword: str,
    festival_title: str,
    festival_start_date: str
) -> List[Dict[str, Any]]:

    # ✔ 오늘 기준 1년 계산
    today = date.today()
    one_year_ago = today - timedelta(days=365)

    # Google Trends 날짜 형식: "YYYY-MM-DD YYYY-MM-DD"
    timeframe = f"{one_year_ago.strftime('%Y-%m-%d')} {today.strftime('%Y-%m-%d')}"

    try:
        # 페이로드 생성 (메인 키워드만)
        _PT.build_payload(
            kw_list=[keyword],
            timeframe=timeframe,
            geo="KR"
        )

        # 데이터 요청
        iot = None
        for i in range(4):
            try:
                iot = _PT.interest_over_time()
                break
            except Exception as e:
                if "429" in str(e):
                    time.sleep((2 ** i) + random.uniform(0, 0.5))
                    continue
                print("❗ Google Trends 오류:", e)
                return []

        if iot is None or iot.empty:
            return []

        # isPartial 제거
        iot = iot.drop(
            columns=[c for c in iot.columns if "ispartial" in str(c).lower()],
            errors="ignore"
        )

        # 메인 키워드 기준 데이터 추출
        series = iot.iloc[:, 0]

        # period/ratio 변환
        graph_data = []
        for idx, value in series.items():
            graph_data.append({
                "period": idx.strftime("%Y-%m-%d"),
                "ratio": int(value) if pd.notnull(value) else 0
            })

        return graph_data

    except Exception as e:
        print("❗ Google Trends 전체 오류:", e)
        return []




=== Google Trends 정확한 1년치 테스트 ===
[{'period': '2024-11-17', 'ratio': 17}, {'period': '2024-11-24', 'ratio': 24}, {'period': '2024-12-01', 'ratio': 30}, {'period': '2024-12-08', 'ratio': 34}, {'period': '2024-12-15', 'ratio': 49}, {'period': '2024-12-22', 'ratio': 100}, {'period': '2024-12-29', 'ratio': 9}, {'period': '2025-01-05', 'ratio': 5}, {'period': '2025-01-12', 'ratio': 3}, {'period': '2025-01-19', 'ratio': 3}]
총 개수: 53


c:\final_project\ACC\acc-ai\.venv\Lib\site-packages\pytrends\request.py:259: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(False)


In [4]:
print(data)

[{'period': '2024-11-17', 'ratio': 17}, {'period': '2024-11-24', 'ratio': 24}, {'period': '2024-12-01', 'ratio': 30}, {'period': '2024-12-08', 'ratio': 34}, {'period': '2024-12-15', 'ratio': 49}, {'period': '2024-12-22', 'ratio': 100}, {'period': '2024-12-29', 'ratio': 9}, {'period': '2025-01-05', 'ratio': 5}, {'period': '2025-01-12', 'ratio': 3}, {'period': '2025-01-19', 'ratio': 3}, {'period': '2025-01-26', 'ratio': 3}, {'period': '2025-02-02', 'ratio': 2}, {'period': '2025-02-09', 'ratio': 2}, {'period': '2025-02-16', 'ratio': 2}, {'period': '2025-02-23', 'ratio': 2}, {'period': '2025-03-02', 'ratio': 2}, {'period': '2025-03-09', 'ratio': 2}, {'period': '2025-03-16', 'ratio': 2}, {'period': '2025-03-23', 'ratio': 2}, {'period': '2025-03-30', 'ratio': 2}, {'period': '2025-04-06', 'ratio': 2}, {'period': '2025-04-13', 'ratio': 1}, {'period': '2025-04-20', 'ratio': 2}, {'period': '2025-04-27', 'ratio': 2}, {'period': '2025-05-04', 'ratio': 2}, {'period': '2025-05-11', 'ratio': 2}, {'pe

In [2]:
if __name__ == "__main__":
    print("=== Google Trends 1년 분석 테스트 ===")

    keyword = "크리스마스"
    festival_title = "담양 산타 축제"
    festival_start_date = "2025-12-24"

    data = get_google_trends_1year(keyword, festival_title, festival_start_date)

    print("\n[Google 1년 트렌드 그래프 데이터]")
    print(data[:10])  # 앞 10개만 출력

    print(f"\n총 데이터 개수: {len(data)}")

=== Google Trends 1년 분석 테스트 ===

[Google 1년 트렌드 그래프 데이터]
[{'period': '2024-11-17', 'ratio': 17}, {'period': '2024-11-24', 'ratio': 24}, {'period': '2024-12-01', 'ratio': 30}, {'period': '2024-12-08', 'ratio': 34}, {'period': '2024-12-15', 'ratio': 49}, {'period': '2024-12-22', 'ratio': 100}, {'period': '2024-12-29', 'ratio': 9}, {'period': '2025-01-05', 'ratio': 5}, {'period': '2025-01-12', 'ratio': 3}, {'period': '2025-01-19', 'ratio': 3}]

총 데이터 개수: 53


c:\final_project\ACC\acc-ai\.venv\Lib\site-packages\pytrends\request.py:259: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(False)
